In [4]:
import numpy as np

In [6]:
def MC(s0:float, k:float, r:float, q:float, sigma:float, T:float, 
       timestep:int, epoch:int, call_option:bool, seednum:int) -> float:
    '''
    Implementation of monte-carlo simulation, assuming each increment/step
    (change of price) is normally distributed, the drift and variance rate of
    x remain constant, equal to their values at time t, and in the time interval 
    of delta t.
    @param s0: spot price for the stock
    @param k: strike price
    @param r: risk free rate
    @param q: dividend yield
    @param sigma: volatility
    @param T: time span
    @param timestep: number of steps within each time span
    @param epoch: number of paths
    @param call_option: if a call or put option
    @param seednum: random seed

    @return out: list of simulated price of dimension epoch
    '''
    np.random.seed(seednum)

    mu = r - q
    var = sigma ** 2
    dt = T / timestep
    out = []

    # epoch start
    for e in range(epoch):

        # start simulation of each path
        dx = 0
        for _ in range(timestep):
            
            # start each dt
            dx += (mu - var/2)*dt + sigma*np.random.normal()*np.sqrt(dt)
        
        # add to the log of s0
        st = np.log(s0) + dx
        st = np.exp(st)
        
        # calculate the value of the option regarding st
        out.append(max(st-k, 0)) if call_option else out.append(max(k-st, 0))
    
    return np.mean(out)

In [7]:
s0 = 100
k = 100 
r = 0.06
q = 0.06
sigma = 0.35 
T = 1 
timestep = 100 
epoch = 4000
call_option = True
seednum = 102623

MC(s0, k, r, q, sigma, T, timestep, epoch, call_option, seednum)

14.337652874702002